In [58]:
import pandas as pd


filename='meteorites.csv'

df = pd.read_csv(filename)

df = df.sort_values(by=['Acquisition Date: (Acquisition)'], ascending=True)

impacts=len(df)
print(impacts)

df.head()

175


,Prefix: (Registration Details),Reg No: (Registration Details),Suffix: (Registration Details),Site: (Collection Details)/Site: (Object Data from time of Collecting Event)/Event/Site: (Object Data from time of Collecting Event),Name: (Geological Details),Geo. Details/ Synonym: (Geological Details),Total Weight: (Specimen Details),Type: (Geological Details),Distribution,Acquisition Date: (Acquisition)
134,MET,135,NaN,"Australia, Western Australia, Kalgoorlie distr...",NaN,NaN,unknown,Stone. Chondrite,SAM: MET135A 58.8 g.,Apr 1945
146,MET,147,NaN,"Australia, Western Australia, Nullarbor or Kal...",NaN,NaN,NaN,Iron.,SAM: MET147A 69 g,Apr 1945
132,MET,133,NaN,"Australia, Western Australia, Kalgoorlie distr...",NaN,NaN,unknown,Stone. Chondrite,SAM: MET133A 346 g.,Apr 1945
133,MET,134,NaN,"Australia, Western Australia, Kalgoorlie distr...",NaN,NaN,unknown,Stone. Chondrite,SAM: MET134A 142.7 g.,Apr 1945
135,MET,136,NaN,"Australia, Western Australia, Kalgoorlie distr...",NaN,NaN,unknown,Stone. Chondrite,SAM: MET136A 81.3 g.,Apr 1945


In [59]:
print(df.columns)
print(df.dtypes)

Index(['Prefix: (Registration Details)', 'Reg No: (Registration Details)',
       'Suffix: (Registration Details)',
       'Site: (Collection Details)/Site: (Object Data from time of Collecting Event)/Event/Site: (Object Data from time of Collecting Event)',
       'Name: (Geological Details)',
       'Geo. Details/ Synonym: (Geological Details)',
       'Total Weight: (Specimen Details)', 'Type: (Geological Details)',
       'Distribution', 'Acquisition Date: (Acquisition)'],
      dtype='object')
Prefix: (Registration Details)                                                                                                           object
Reg No: (Registration Details)                                                                                                            int64
Suffix: (Registration Details)                                                                                                           object
Site: (Collection Details)/Site: (Object Data from time of Colle

In [63]:
import plotly_express as px

id = df['Reg No: (Registration Details)'].values
type = df['Type: (Geological Details)'].values

fig = px.scatter(df,x=id,y=type,labels={'x':'Reg No: (Registration Details)', 'y':'Type: (Geological Details)'})
fig.show()

time_yrs = max(id)-id

fig_2 = px.scatter(df,x=time_yrs,y=type, labels={'x':' Reg No: (Registration Details) since impact 0', 'y':'Type: (Geological Details)'})
fig_2.show()

In [64]:
def map_value(value, min_value, max_value, min_result, max_result):
    '''maps value (or array of values) from one range to another'''
    
    result = min_result + (value - min_value)/(max_value - min_value)*(max_result - min_result)
    return result

map_value(1, 0, 1, 0, 100)

100.0

In [68]:
myrs_per_beat = 5 #number of Myrs for each beat of music 

t_data = time_yrs/myrs_per_beat #rescale time from Myrs to beats


duration_beats = max(t_data)  #duration in beats (actually, onset of last note)
print('Duration:', duration_beats, 'beats')



Duration: 34.8 beats


In [69]:
#calculate duration in seconds
import plotly.express as px
bpm = 60  #if bpm = 60, 1 beat = 1 sec 
duration_sec = duration_beats*60/bpm #duration in seconds (actually, onset of last note)
print('Duration:', duration_sec, 'seconds')


duration = px.scatter(df, x=t_data, y=type, labels={'x': 'time [beats]', 'y': 'Type: (Geological Details)'})
duration.show()

Duration: 34.8 seconds


In [78]:

y_data = map_value(id, min(id), max(id), 0, 1) #normalize data, so it runs from 0 to 1 

y_scale = 0.5  #lower than 1 to spread out more evenly

y_data = y_data**y_scale

normalized_data = px.scatter(df, x=time_yrs,y=y_data,labels={'x':'Reg No: (Registration Detials)','y':'y_data normalized'})
normalized_data.show()


In [79]:
MIDI_A4 = 69   # MIDI Pitch number
import itertools as it
from numpy import nan


def str2midi(note_string):
  """
  Given a note string name (e.g. "Bb4"), returns its MIDI pitch number.
  """
  if note_string == "?":
    return nan
  data = note_string.strip().lower()
  name2delta = {"c": -9, "d": -7, "e": -5, "f": -4, "g": -2, "a": 0, "b": 2}
  accident2delta = {"b": -1, "#": 1, "x": 2}
  accidents = list(it.takewhile(lambda el: el in accident2delta, data[1:]))
  octave_delta = int(data[len(accidents) + 1:]) - 4
  return (MIDI_A4 +
          name2delta[data[0]] + # Name
          sum(accident2delta[ac] for ac in accidents) + # Accident
          12 * octave_delta # Octave
         )


In [83]:
#from audiolazy import str2midi #import function to convert note names to midi numbers (could also use: midi2str, str2freq, freq2str,freq2midi,midi2freq) https://pypi.org/project/audiolazy/

# 4 octaves of major scale
note_names = ['C2','D2','E2','F2','G2','A2','B2',
             'C3','D3','E3','F3','G3','A3','B3',
             'C4','D4','E4','F4','G4','A4','B4',
             'C5','D5','E5','F5','G5','A5','B5']

#4 octaves of major pentatonic scale 
note_names = ['C2','D2','E2','G2','A2',
             'C3','D3','E3','G3','A3',
             'C4','D4','E4','G4','A4',
             'C5','D5','E5','G5','A5']

#custom note set (a voicing of a Cmaj13#11 chord, notes from C lydian)
note_names = ['C1','C2','G2',
             'C3','E3','G3','A3','B3',
             'D4','E4','G4','A4','B4',
             'D5','E5','G5','A5','B5',
             'D6','E6','F#6','G6','A6']

note_midis = [str2midi(n) for n in note_names] #make a list of midi note numbers 

n_notes = len(note_midis)
print('Resolution:',n_notes, 'notes')

note_midis[10]


Resolution: 23 notes


67

In [85]:
midi_data = []
for i in range(impacts):
    note_index = round(map_value(y_data[i], 0, 1, n_notes-1, 0)) #notice choice of polarity: bigger craters are mapped to lower notes
                                                        #we round the result because it's a list index which must be an integer
    midi_data.append(note_midis[note_index])
    

midi_note_num = px.scatter(df,x=t_data, y=midi_data,labels={'x':'time [beats]', 'y':'midi note numbers'})
midi_note_num.show()


In [86]:
vel_min,vel_max = 35,127   #minimum and maximum note velocity

vel_data = []
for i in range(impacts):
    note_velocity = round(map_value(y_data[i], 0, 1, vel_min, vel_max)) #bigger craters will be louder
                                                                #we round here because note velocites are integers
    vel_data.append(note_velocity)
    
velocity = px.scatter(df, x=t_data, y=midi_data,labels={'x':'time [beats]', 'y':'midi note numbers'})
velocity.show()


In [87]:
from midiutil import MIDIFile #import library to make midi file, https://midiutil.readthedocs.io/en/1.2.1/
    
#create midi file object, add tempo
my_midi_file = MIDIFile(1) #one track 
my_midi_file.addTempo(track=0, time=0, tempo=bpm) 

#add midi notes
for i in range(impacts):
    my_midi_file.addNote(track=0, channel=0, pitch=midi_data[i], time=t_data[i], duration=2, volume=vel_data[i])

#create and save the midi file itself
with open(filename + '.mid', "wb") as f:
    my_midi_file.writeFile(f) 
